In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.action_chains import ActionChains 
import json
import csv

In [53]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--headless")
options.add_argument("user-agent=Chrome/90.0.4430.97 Safari/537.36")
driver = webdriver.Chrome(options=options, executable_path="./chromedriver")

In [54]:
# Scroll page to load whole content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [55]:
comments_list = []
first_url = "https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=1"
driver.get(first_url)
comment_url=driver.find_elements_by_css_selector(".c-pager__item")[2].get_attribute("href")
url_format = comment_url[:-1]
print(url_format)

# driver.get(url_format+str(200))
# last_comment_pager=driver.find_elements_by_css_selector(".c-pager__item")[-1].get_attribute("href")
# last_number = int(last_comment_pager.replace(url_format, ""))
last_number = 6
print(last_number)
for i in range(1, last_number+1):
    url = url_format+str(i)
    driver.get(url)
    time.sleep(2)
    if driver.find_elements_by_css_selector(".c-comments__content") == []:
        print("Network Error")
        i -= 1
        continue
    print(url)
    comment_text = driver.find_elements_by_css_selector(".c-comments__content")
    comment_detail = driver.find_elements_by_css_selector(".c-comments__detail")
    for j in range(0,len(comment_text)):
        comment_dict = {
            "date":comment_detail[j].text,
            "auther":comment_detail[j+1].text,
            "text":comment_text[j].text,
            "polarity":0
        }
        comments_list.append(comment_dict)

https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=
6
https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=1
https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=2
https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=3
https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=4
https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=5
https://www.digikala.com/ajax/product/comments/list/1013318/?mode=newest_comment&page=6


In [56]:
print(len(comments_list))
# for i in range(len(comments_list)):
#     print(comments_list[i].get('text'))
#     pol = input()
#     comments_list[i]['polarity'] = pol

keys = comments_list[0].keys()
with open('amazon_paperwhite.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(comments_list)

78


In [57]:
driver.quit()

# algorithm implementation

## polarity annotation

In [96]:
import pandas as pd

df = pd.read_csv("amazon_paperwhite.csv")
df

,Unnamed: 0,date,auther,text,polarity
0,0,۲۲ آبان ۱۴۰۰,آتنا دهقان,با سلام. بعد از یک هفته استفاده دستگاه هنگ کرد...,0
1,1,آتنا دهقان,۱۹ آبان ۱۴۰۰,متاسفانه نسخه با تبلیغات را دریافت کردم . تبلی...,0
2,2,۱۹ آبان ۱۴۰۰,کاربر دیجی‌کالا,در کل بسیار خوبه...مخصوصا برای متن های انگلیسی...,0
3,3,کاربر دیجی‌کالا,۱۶ آبان ۱۴۰۰,برای استادو دانشجو و هر کسی که دنباله دانشو عل...,0
4,4,۱۶ آبان ۱۴۰۰,عاطفه امیری,فضای مطالعه با تبلت و موبایل بسیار متفاوت بوده...,0
...,...,...,...,...,...
73,73,مهرداد نیازی,۱۰ آبان ۱۳۹۸,کتابخوان کیندل محدودیت دسترسی به کتابهای خارجی...,0
74,74,۱۰ آبان ۱۳۹۸,محمد سلطانی,اگر اهل کتاب خوندن هستید این نسل خیلی عالیه هم...,0
75,75,۳۱ اردیبهشت ۱۳۹۸,هومن بلورچی,با فارسی مشکل نداره یه بنده خدایی هم یه پچ فار...,0
76,76,هومن بلورچی,۱۸ فروردین ۱۳۹۸,برای کسی که زیاد کتاب می‌خونه و از حمل کتاب‌ها...,0


In [98]:
for i in range(df.shape[0]):
    print(i, df.at[i, 'text'])
    pos = input()
    df.at[i, 'polarity'] = pos

0 با سلام. بعد از یک هفته استفاده دستگاه هنگ کرد و متاسفانه قابل تعمیر نبود، ا ز طرف فروشگاه و دیجی کالا هم ساپورتی صورت نگرفت. طی تما با فروشنده، برخورد ایشان هم نامناسب و توهین آمیز بود.
2
1 متاسفانه نسخه با تبلیغات را دریافت کردم . تبلیغات آن آزار دهنده است و مرتب سازی کتابخانه را با مشکل مواجه می کند . برنامه اصلی آن هم در هنگام باز کردن کتابها با خطا مواجه می شود . اکانت آمازون امکان ثبت شماره موبایل نمی دهد و امکانات آن شامل کالکشن ها فعال نمی شود. بعد میگن سرانه کتابخوانی پایین هست سپاس
2
2 در کل بسیار خوبه...مخصوصا برای متن های انگلیسی. البته بعضی فایل های فارسی راهم خیلی خوب نمایش میده.در حالت افقی متن را با انازه کتاب نشان میده. PDF فارسی و سایر فایل های انگلیسی را به خوبی پوشش می دهد. حافظه داخلی 6 گیگ داره...که واقعا کافیه. با USB که به کامپیوتر وصل کنید..میتونین فایل های مورد نظرتون را به قسمت document منتقل کنین. باwifi به اینترنت وصل میشه و اگر حساب کاربری آمازون داشته باشین.. میتونین از کتاب های آمازون هم استفاده کنین. در ضمن چه در روز و چه در شب چشم را اذیت نمی کنه.
1


1
20 با سلام و خسته نباشید همه چیز عالی فقط زمان تحویل با یک روز تاخیر
1
21 وسیله پرکاربرد برای افراد کتاب خوان که باعث کاهش هزینه خرید کتاب و صرفه جویی در مصرف کاغذ میشه. بنده استفاده کردم، به راحتی میشه فایل PDF کتاب ها رو اد کرد و استفاده نمود.
1
22 کتاب فارسی با فرمت کیندل پیدا نمیشه باید پی دی اف خوند
2
23 حرف نداره ای هر نظر. خیلی با کتاب خوندن توی گوشی فرق داره البته برای کسایی که به زبان غیر فارسی کتاب میخونن، فارسی هم پی دی اف میخونه. در کل صفحه‌ش مثل کتاب می‌مونه و واقعا هم زیباست.
1
24 یک کتابخوان عالی که اگر قیمت ریالی آن مناسب بود می شد به همه پیشنهاد کرد. نسل ۱۰ رنگبندی های خوبی دارد که من رنگ sage را خریدم. در ضمن هزینه اضافه بابت نسخه بدون تبلیغات ندین چون هیچ مزاحمتی ایجاد نمی کنه. هر از گاهی یه کتاب تو لاک اسکرین نشون میده که جالب هم هست و ممکنه کتاب جدید هم پیدا کنین.
1
25 دوستان من دو سه هفته هست که خریدم و و از وقتی زدم شارژ و هر روز حدود نیم ساعت کار میکنم باهاش تقریبا چهل درصد کم کرده و الان شصت درصد شارژ داره و واقعا وقتی نوشته یک ماه نگه میداره مطمئن باشید حتی 

2
57 عالیه. اگه ادمی هستید که کتاب الکترونیک زیاد میخونید خب کیندل یه ضرورت برای محافظت از چشم هاتون. من اول این کالا رو از ملیز خریداری کردم، ولی مرجوع کردم چون خیلی زود باتری تموم میکرد و صفحه نمایش پرش داشت. مجددا از شمس خریداری کردم، همه چیش عالی بود. مثل مشخصات.
1
58 اگر کتاب انگلیسی می خوانید حتما این محصول را تهیه کنید و لذت خواندن بدون خستگی چشم را تجربه کنید.
1
59 عالیه ولی با این قیمت به هیچ عنوان نمی ارزه
2
60 خیلی گرون شده ارزش خرید نداره
2
61 اگر قصد خرید کیندل دارید حتما یک بار هم شده از کیندل دوستانتون استفاده کنید یا ویدیو‌هاشو ببینید تا بدونید با چه محصولی مواجهید. اگر اهل مطالعه‌ی زیاد هستید تکنولوژی E-Ink فرشته‌ی نجات چشمان شما دربرابر صفحات مانیتور و موبایل هست. یک کاغذی که تعاملی(interactive) است.با یک بار شارژ تا یک ماه میتونید از کیندل استفاده کنید و میتونید کتاب‌های مورد نظرتون رو با نرم‌افزار calibre از epub به راحتی به mobi که فرمت محلی کیندله تبدیل کنید. (البته از pdf هم پشتیبانی میکنه) درکل کیندل مناسب کساییه که درهمه حالت میخوان کتاب بخونند و حجم زیاد کتابا

In [99]:
df['polarity'] = df['polarity'].replace(2,'-1').replace(1, '+1').replace(-1,'-1')
df.drop(df[df['polarity'] == 0].index, inplace=True)
df['polarity'].value_counts()

+1    51
-1    27
Name: polarity, dtype: int64

In [100]:
df.to_csv('amazon_paperwhite.csv')

## creating sentence-based corpus

In [101]:
data = df = pd.read_csv("amazon_paperwhite.csv")
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data['polarity'].value_counts()

 1    51
-1    27
Name: polarity, dtype: int64

In [102]:
from parsivar import Tokenizer
from parsivar import Normalizer
my_normalizer = Normalizer()
my_tokenizer = Tokenizer()

sentence_list = []
for i in range(data.shape[0]):
    text = data.at[i, 'text']
    sent_tokens = my_tokenizer.tokenize_sentences(my_normalizer.normalize(text))
    sentence_list += sent_tokens
len(sentence_list)

257

In [103]:
sentence_data = []
for i in range(len(sentence_list)):
    print(i, sentence_list[i])
    pol = input()
    sentence_dict = {
        'text': sentence_list[i],
        'polarity': pol
    }
    sentence_data.append(sentence_dict)

0 با سلام  .
0
1  بعد از یک هفته استفاده دستگاه هنگ کرد و متاسفانه قابل تعمیر نبود ، ا ز طرف فروشگاه و دیجی کالا هم ساپورتی صورت نگرفت  .
2
2  طی تما با فروشنده ، برخورد ایشان هم نامناسب و توهین آمیز‌بود  .
2
3 متاسفانه نسخه با تبلیغات را دریافت کردم  .
2
4  تبلیغات آن آزار‌دهنده است و مرتب‌سازی کتابخانه را با مشکل مواجه می‌کند  .
2
5  برنامه اصلی آن هم در هنگام باز کردن کتابها با خطا مواجه می‌شود  .
2
6  اکانت آمازون امکان ثبت شماره موبایل نمی‌دهد و امکانات آن شامل کالکشن‌ها فعال نمی‌شود  .
2
7  بعد میگن سرانه کتابخوانی پایین هست سپاس
2
8 در کل بسیار خوبه  ...
1
9  مخصوصا برای متن‌های انگلیسی  .
1
10  البته بعضی فایل‌های فارسی راهم خیلی خوب نمایش میده  .
2
11  در حالت افقی متن را با انازه کتاب نشان میده  .
1
12  PDF فارسی و سایر فایل‌های انگلیسی را به خوبی پوشش می‌دهد  .
1
13  حافظه داخلی 6 گیگ داره  ...
1
14  که واقعا کافیه  .
1
15  با USB که به کامپیوتر وصل کنید  ..
1
16  میتونین فایل‌های مورد نظرتون را به قسمت document منتقل کنین  .
1
17  باwifi به اینترنت وصل میشه و اگر حساب کاربر

1
86  البته برای بقیه ی فرمت‌ها مثل epub هنوز این روش رو امتحان نکردم ولی برای پی دی اف این روش جواب میده و نیازی هم نیست حتما پی دی اف رو جداگانه به چیز دیگه‌ای تبدیل کنید  .
1
87  فایل‌های epub هم با نرم‌افزار قابل تبدیل شدن به mobi رو دارن و کار آسونیه  .
1
88  شارژ رو نسبتا خوب نگه می‌داره ولی فکر نکنم اونجور که میگن برای چند هفته نگه داره ، نهایتا یک هفته  .
1
89  البته بستگی به استفاده هم داره و هرچی بیشتر از وای فای استفاده کنید زودتر شارژ تموم میکنه  .
1
90  برای یادگیری زبان و خوندن کتاب انگلیسی فوق العاده مفیده  .
1
91  دیکشنری داره که می‌تونید موقع خوندن اگه معنی کلمه‌ای رو نمی‌دونستید روش نگه دارید و معنیش رو خودش از دیکشنری پیدا میکنه و نشون میده  .
1
92  بعد کلماتی رو که سرچ کردید ذخیره میکنه با اون جمله ی متن که این کلمه توش‌بود و خودش براتون فلش کارت هم درست میکنه  .
1
93  تو قسمت vocabulary builder این امکان رو داره  .
1
94  کلا برای انگلیسی خوندن خیلی مفیده ولی برای فارسی چندان مطمئن نیستم  .
1
95  شاید برای کسی که بیشتر دنبال مطالعه به زبان فارسیه بهتر باشه بیشتر به 

1
181  برای متون فارسی هم فایلهای epub به AZW3 تبدیل کردم و فایلهای pdf بهینه کردم  .
1
182 قیمت خیلی خافتضاح
2
183 راستش من به شدت مشتاق‌بودم این محصول رو برای مطالعه کتاب‌های زبان اصلی بخرم برای همین نزدیک به یک ماه به دیجی سر میزدم و قیمتشو چک میکردم و شاهد افزایش متناوب و بی‌دلیلش‌بودم تا جایی که اعتماد خریدمو از دست دادم  !
2
184 چون در حال یادگیری زبان هستم این کتابخوان رو تهیه کردم و واقعا محصول بااصالت و باکیفیتی هست اونایی که میگن بده باید هرچیزی رو تو حوزه خودش مقایسه کرد  ..
1
185  اگه شما با تبلت‌های گرونم مقایسش کنید قطعا تبلت نمیتونه جای اینو بگیره و این هم نمیتونه جای تبلت رو بگیره واقعا سبک و خوش دست هست و من کتابهای فارسی هم ریختم تا الان که مشکلی نداشتم ولی اگه انتظارات دیگه ازش دارین اصلا اصلا سراغش نرین برای اهل مطالعه و کسایی که دوس دارن کتابهای روز انگلیسی رو بخونن عالی هست و اونایی که میگن قیمتش بالاست بعنوان یه محصول امریکایی از خیلی از تبلت‌های معمولی کره‌ای و چینی ارزون تر هست
1
186 قیمت این کتابخوان در سایت آمازون 130 دلار  .
1
187  قیمت اینجا مساوی پول یه لپ

1
244  خلاصه اگه اهل مطالعه هستید بخرید حتما هم بخرید تاکید میکنم اهل مطالعه وگرنه پشیمون میشید
1
245 با فارسی مشکل نداره یه بنده خدایی هم یه پچ فارسی براش نوشته که مشکلات جزیی با فارسی اگرم باشه برطرف میشه که نداشته تا حالا  ...
1
246  فقط اگه زبان اصلی کتاب میخونید عالیه ، من خودم فقط زبان اصلی میخونم و رضایت 100 درصد دارم  ..
1
247  میشه تقریبا تمام کتاب سایت امازون با فرمت امازون تو اینترنت رایگان ( به لطف نبود قانون کپی رایت ) پیدا کرد و روش نصب کرد ولی خب کتابا فارسی اغلب با فرمت امازون کانورد‌نشده‌یا به صورت فایل وورد و  ...
1
248  نیست و pdf هم فقط در صورتی خوب جواب میده که زیاد فونتش کوچیک نباشه چون اگه ادم بخواد هی زوم کنه رو صفحه جابجا بشه رو اعصابه نداشتن هدف و  ...
1
249  مزیت نه عیب چون با فلسفه کتاب خوندن مغایرت داره
1
250 برای کسی که زیاد کتاب می‌خونه و از حمل کتاب‌های سنگین خسته‌شده‌، کتابخوان الکترونیک چیزیه که یه روزی بالاخره به خریدش فکر می‌کنه  .
1
251  اما نداشتن حس و حال خوب کتاب کاغذی تنها مشکل با این محصول نیست  .
2
252  متاسفانه خیلی کندتر از چیزیه که با پیشرف

In [104]:
len(sentence_data)

257

In [105]:
keys = sentence_data[0].keys()
with open('amazon_paperwhite_sentence.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sentence_data)

In [135]:
new_data =  pd.read_csv("../data/fidibook_hannah_sentence.csv")
new_data['polarity'].value_counts()

1    247
2    127
0     54
Name: polarity, dtype: int64

In [107]:
d = pd.read_csv("amazon_paperwhite.csv")
d['polarity'].value_counts()

 1    51
-1    27
Name: polarity, dtype: int64

## dataset features

In [126]:
df = pd.read_csv("../data/fidibook_hannah.csv")
df

,Unnamed: 0,date,auther,text,polarity
0,1,محمد باباجانپور عزیزی,۶ آبان ۱۴۰۰,سلام این کتابخوان توسط یک برند چینی طراحی و سا...,1
1,6,۲ آبان ۱۴۰۰,محمد دوستان,چیزی که تو ذوق من زد اندازه خیلی کوچکتر از تصو...,-1
2,9,سهراب عطایی,۲۸ مهر ۱۴۰۰,شش ماهه هر روز ازش استفاده می‌کنم و راضی هستم،...,1
3,10,۲۸ مهر ۱۴۰۰,کاربر دیجی‌کالا,حرف نداره علی الخصوص برای مسافرت و مطالعه بیرون,1
4,13,مرضیه رحیمیان,۲۲ مهر ۱۴۰۰,خیلی راضی هستم از خریدش دوتا هم خریدم ازش. اگر...,1
...,...,...,...,...,...
121,149,۱۵ اردیبهشت ۱۴۰۰,بهرخ بختیاری,این کتابخوان با اندازه ی مناسب به راحتی قابل ح...,1
122,150,۸ اردیبهشت ۱۴۰۰,کاربر دیجی‌کالا,باحاله. کار باهاش یکم صبر میخواد,-1
123,151,کاربر دیجی‌کالا,۸ اردیبهشت ۱۴۰۰,من از خریدم واقعا راضی هستم.,1
124,152,۸ اردیبهشت ۱۴۰۰,حمیدرضا سرپولکی,عالی ترین خودش هست,1


In [127]:
from hazm import *
number_tokens = 0
print(df.shape[0])
for i in range(df.shape[0]):
    text = df.at[i, 'text']
    tokenizer = WordTokenizer()
    comment = tokenizer.tokenize(text)
    number_tokens += len(comment)
    
print(number_tokens)

126
7778


In [129]:
number_tokens/df.shape[0]

61.73015873015873